In [345]:
# !pip3 install pypdf2

In [346]:
from PyPDF2 import PdfReader
reader=PdfReader("Freakonomics 1.pdf","rb")

In [347]:
length=len(reader.pages)
pages=[]

In [348]:
for i in range(length):
    page=PdfReader._get_page(reader,page_number=i)
    pages.append(page)

In [349]:
texts=[]
for i in range(length):
    text=pages[i].extract_text()
    texts.append(text)

In [350]:
print(texts[55])

FREAKONOMICS 
tournaments held immediately after allegations of match rigging, 7–7 
wrestlers win only 50 percent of their ﬁnal-day matches against 8–6 opponents instead of the typical 80 percent. No matter how the data 
are sliced, they inevitably suggest one thing: it is hard to argue that 
sumo wrestling isn’t rigged. 
Several years ago, two former sumo wrestlers came forward with 
extensive allegations of match rigging—and more. Aside from the crooked matches, they said, sumo was rife with drug use and sexca
-
pades, bribes and tax evasion, and close ties to the yakuza , the Japa -
nese maﬁa. The two men began to receive threatening phone calls; one of them told friends he was afraid he would be killed by the yakuza . Still, they went forward with plans to hold a press conference 
at the Foreign Correspondents’ Club in Tokyo. But shortly before
-
hand, the two men died—hours apart, in the same hospital, of a sim -
ilar respiratory ailment. The police declared there had been no foul

In [351]:
import pandas as pd

In [352]:
contents=pd.DataFrame(columns=["pageNumber","content"])

In [353]:
data=[]
for num,val in enumerate(texts):
    data.append({"pageNumber":num,"content":val})
contents=pd.concat([contents,pd.DataFrame(data)],ignore_index=True)

In [354]:
contents.head()

,pageNumber,content
0,0,
1,1,
2,2,FREAKONOMICS A Rogue Economist \nExplores the ...
3,3,
4,4,CONTENTS \nAN EXPLANATORY NOTE vii \nIn which ...


In [355]:
contents

,pageNumber,content
0,0,
1,1,
2,2,FREAKONOMICS A Rogue Economist \nExplores the ...
3,3,
4,4,CONTENTS \nAN EXPLANATORY NOTE vii \nIn which ...
...,...,...
335,335,"Index \nSwitzerland, 224–25 \nSzilagyi, John, ..."
336,336,About the Authors \nSTEVEN D. L EVITT teaches...
337,337,Credits \nJacket design by Chika Azuma \nJack...
338,338,Copyright \nFREAKONOMICS (REVISED AND EXPANDED...


In [356]:
contents.describe()

,pageNumber,content
count,340,340
unique,340,330
top,0,
freq,1,11


In [357]:
content.head()

,pageNumber,content
0,0,
1,1,
2,2,FREAKONOMICS A Rogue Economist \nExplores the ...
3,3,
4,4,CONTENTS \nAN EXPLANATORY NOTE vii \nIn which ...


In [358]:
import spacy
nlp=spacy.load("en_core_web_sm")

In [359]:
contents.content=contents["content"].apply(nlp)

In [363]:
contents.head()

,pageNumber,content
0,0,()
1,1,()
2,2,"(FREAKONOMICS, A, Rogue, Economist, \n, Explor..."
3,3,()
4,4,"(CONTENTS, \n, AN, EXPLANATORY, NOTE, vii, \n,..."


In [364]:
def remove_newline(doc):
    return[token.text.replace("\n","") for token in doc]
contents.content=contents["content"].apply(remove_newline)

In [368]:
contents.head()

,pageNumber,content
0,0,[]
1,1,[]
2,2,"[FREAKONOMICS, A, Rogue, Economist, , Explores..."
3,3,[]
4,4,"[CONTENTS, , AN, EXPLANATORY, NOTE, vii, , In,..."


In [360]:
import pandas as pd

In [361]:
texts[5]

'Co n t e n t s \nWho cheats? Just about everyone...H o w cheaters cheat, and how to \ncatch them ...S tories from an Israeli day-care cente r... The sudden dis -\nappearance of seven million American childre n...C heating schoolteachers \nin Chicag o...W h y cheating to lose is worse than cheating to wi n... \nCould sumo wrestling, the national sport of Japan, be corrupt ?...W h a t \nthe Bagel Man saw: mankind may be more honest than we think. \n2. How Is the Ku Klux Klan Like a Group \nof Real-Estate Agents? 49 \nIn which it is argued that nothing is more powerful than information, \nespecially when its power is abused. \nSpilling the Ku Klux Klan’s secret s...W h y experts of every kind are in \nthe perfect position to exploit yo u... The antidote to information abuse: \nthe Interne t...W h y  a new car is suddenly worth so much less the moment \nit leaves the lo t...B r eaking the real-estate agent code: what “well main -\ntained” really mean s...I s  T r ent Lott more racist than

In [362]:
import re
pattern="\"


SyntaxError: EOL while scanning string literal (1773421775.py, line 2)